In [1]:
import numpy as np
import pandas as pd
from DecisionTree import DecisionTree
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

%run DecisionTree.py


In [2]:
# Load dataset
data = pd.read_csv('heart.csv')
# Binary encoding for 'Sex' and 'ExerciseAngina'
data['Sex'] = data['Sex'].map({'M': 1, 'F': 0})
data['ExerciseAngina'] = data['ExerciseAngina'].map({'Y': 1, 'N': 0})
# Extract features and labels
X = data.drop(columns=['HeartDisease'])
y = data['HeartDisease']

In [ ]:
# One-Hot Encoding for non-binary categorical features
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
encoder = OneHotEncoder(sparse=False)

if categorical_cols:  # Only encode if there are categorical features
    encoded_array = encoder.fit_transform(X[categorical_cols])
    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols))
    X = pd.concat([X.drop(columns=categorical_cols), encoded_df], axis=1)

X = X.to_numpy()
y = y.to_numpy()

In [4]:
# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

print(X_train.shape, X_val.shape, X_test.shape)

(642, 18) (92, 18) (184, 18)


In [5]:
# Train the DecisionTree model
model = DecisionTree()
model.fit(X_train, y_train)

print("Training Complete!")

Training Complete!


In [31]:
# tune the hyperparameters of the DecisionTree model
max_depths = [5, 10, 15, 20, 25]
min_samples_splits = [2, 4, 6, 8, 10]

# Initialize best hyperparameters
best_max_depth = None
best_min_samples_split = None
best_accuracy = 0

# Tune hyperparameters
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        model = DecisionTree(max_depth=max_depth, min_sample_split=min_samples_split)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        if accuracy > best_accuracy:
            best_max_depth = max_depth
            best_min_samples_split = min_samples_split
            best_accuracy = accuracy
            

# Train the DecisionTree model with the best hyperparameters
model = DecisionTree(max_depth=best_max_depth, min_sample_split=best_min_samples_split)
model.fit(X_train, y_train)


In [32]:
# get predictiona and accuracy of the DecisionTree model on the test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of DecisionTree: {accuracy}")

Accuracy of DecisionTree: 0.7554347826086957


In [33]:
# train sklearn's DecisionTreeClassifier
sklearn_model = DecisionTreeClassifier(random_state=42)
sklearn_model.fit(X_train, y_train)

# predict on validation set
val_set = sklearn_model.predict(X_val)
# compute accuracy
accuracy = accuracy_score(y_val, val_set)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.8587


In [11]:
# train logistic regression
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)

# predict on validation set
val_set = logistic_model.predict(X_val)
# compute accuracy
accuracy = accuracy_score(y_val, val_set)

print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8913


c:\Users\bo2dy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Hyper-parameters 
input_size = 18
hidden_size = 100 
num_classes = 2
num_epochs = 20
batch_size = 64
learning_rate = 0.001

In [34]:
from sklearn.preprocessing import StandardScaler

# Create a scaler and scale the data:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)          
X_val_scaled = scaler.transform(X_val)            

# Convert the scaled NumPy arrays to PyTorch tensors.
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Use torch.long for classification targets.

x_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

x_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Print shapes to verify correct transformation:
print(X_train_tensor.shape)
print(X_train.shape)
print(y_train_tensor.shape)


# Create DataLoaders
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


torch.Size([642, 18])
(642, 18)
torch.Size([642])


In [35]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, learning_rate, num_epochs):
        super(BinaryClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs

        
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        
        self.l2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x


def train_model(model, train_loader, val_loader):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=model.learning_rate)
    best_val_loss = float('inf')
    early_stopping_threshold = 5
    no_improve_epochs = 0

    for epoch in range(model.num_epochs):
        model.train()
        total_loss = 0
        for example, lab in train_loader:
            optimizer.zero_grad()
            output = model(example).squeeze()  # Remove extra dimension if present
            loss = criterion(output, lab.float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        val_loss = validate_model(model, val_loader, criterion)

        print(f"Epoch [{epoch+1}/{model.num_epochs}], Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1

        if no_improve_epochs >= early_stopping_threshold:
            print(f"Early stopping at epoch {epoch+1}")
            break

def validate_model(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for example, lab in val_loader:
            output = model(example).squeeze()
            loss = criterion(output, lab.float())
            total_loss += loss.item()
    return total_loss / len(val_loader)

def predict(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for example, _ in test_loader:
            output = model(example).squeeze()
            predicted = (output >= 0.5).int()  # Threshold at 0.5 to decide between class 0 and 1
            predictions.extend(predicted.numpy())
    return predictions

# ---------------------------
# 4. Hyperparameter Tuning
# ---------------------------
hidden_sizes = [16, 32]           
learning_rates = [0.001, 0.01, 0.1] 
num_epochs = 50                    

best_hidden_size = None
best_learning_rate = None
best_accuracy = 0

for hidden_size in hidden_sizes:
    for lr in learning_rates:
        print(f"\nTuning with hidden_size: {hidden_size}, learning_rate: {lr}")
        model = BinaryClassifier(input_size=X_train_tensor.shape[1],
                                 hidden_size=hidden_size,
                                 learning_rate=lr,
                                 num_epochs=num_epochs)
        train_model(model, train_loader, val_loader)
        y_pred = predict(model, val_loader)
        acc = accuracy_score(y_val, y_pred)
        print(f"Validation Accuracy: {acc:.4f}")
        if acc > best_accuracy:
            best_accuracy = acc
            best_hidden_size = hidden_size
            best_learning_rate = lr

print("\nBest Hyperparameters:")
print(f"Best Hidden Size: {best_hidden_size}")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Validation Accuracy: {best_accuracy:.4f}")



Tuning with hidden_size: 16, learning_rate: 0.001
Epoch [1/50], Loss: 0.6975, Val Loss: 0.6866
Epoch [2/50], Loss: 0.6701, Val Loss: 0.6551
Epoch [3/50], Loss: 0.6265, Val Loss: 0.6252
Epoch [4/50], Loss: 0.6100, Val Loss: 0.5969
Epoch [5/50], Loss: 0.5933, Val Loss: 0.5705
Epoch [6/50], Loss: 0.5553, Val Loss: 0.5437
Epoch [7/50], Loss: 0.5354, Val Loss: 0.5173
Epoch [8/50], Loss: 0.5321, Val Loss: 0.4920
Epoch [9/50], Loss: 0.4820, Val Loss: 0.4709
Epoch [10/50], Loss: 0.4573, Val Loss: 0.4493
Epoch [11/50], Loss: 0.4556, Val Loss: 0.4284
Epoch [12/50], Loss: 0.4165, Val Loss: 0.4105
Epoch [13/50], Loss: 0.4010, Val Loss: 0.3927
Epoch [14/50], Loss: 0.3917, Val Loss: 0.3761
Epoch [15/50], Loss: 0.3668, Val Loss: 0.3613
Epoch [16/50], Loss: 0.3639, Val Loss: 0.3488
Epoch [17/50], Loss: 0.3702, Val Loss: 0.3379
Epoch [18/50], Loss: 0.3513, Val Loss: 0.3284
Epoch [19/50], Loss: 0.3718, Val Loss: 0.3214
Epoch [20/50], Loss: 0.3390, Val Loss: 0.3167
Epoch [21/50], Loss: 0.3455, Val Loss:

In [36]:
# ---------------------------
# 5. Train Final Model
# ---------------------------
# Train the final model using the best hyperparameters found
final_model = BinaryClassifier(input_size=X_train_tensor.shape[1],
                               hidden_size=best_hidden_size,
                               learning_rate=best_learning_rate,
                               num_epochs=num_epochs)
train_model(final_model, train_loader, val_loader)

y_pred = predict(final_model, test_loader)
y_pred = np.array(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print("Training Complete!")

Epoch [1/50], Loss: 0.6920, Val Loss: 0.6551
Epoch [2/50], Loss: 0.6389, Val Loss: 0.6086
Epoch [3/50], Loss: 0.6122, Val Loss: 0.5685
Epoch [4/50], Loss: 0.5629, Val Loss: 0.5351
Epoch [5/50], Loss: 0.5291, Val Loss: 0.5017
Epoch [6/50], Loss: 0.5181, Val Loss: 0.4706
Epoch [7/50], Loss: 0.4613, Val Loss: 0.4441
Epoch [8/50], Loss: 0.4291, Val Loss: 0.4185
Epoch [9/50], Loss: 0.4008, Val Loss: 0.3948
Epoch [10/50], Loss: 0.4348, Val Loss: 0.3746
Epoch [11/50], Loss: 0.3710, Val Loss: 0.3601
Epoch [12/50], Loss: 0.3545, Val Loss: 0.3470
Epoch [13/50], Loss: 0.3707, Val Loss: 0.3362
Epoch [14/50], Loss: 0.4674, Val Loss: 0.3280
Epoch [15/50], Loss: 0.3391, Val Loss: 0.3288
Epoch [16/50], Loss: 0.3285, Val Loss: 0.3254
Epoch [17/50], Loss: 0.3678, Val Loss: 0.3196
Epoch [18/50], Loss: 0.3643, Val Loss: 0.3143
Epoch [19/50], Loss: 0.4568, Val Loss: 0.3099
Epoch [20/50], Loss: 0.3162, Val Loss: 0.3058
Epoch [21/50], Loss: 0.3484, Val Loss: 0.3018
Epoch [22/50], Loss: 0.3204, Val Loss: 0.29